In [28]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Folders
bw_folder = "./training_data/filtered_data/Input_2"
color_input_folder = "./training_data/filtered_data/Input_1"
color_output_folder = "./training_data/filtered_data/Output"

# List files
bw_files = sorted(os.listdir(bw_folder))
color_input_files = sorted(os.listdir(color_input_folder))
color_output_files = sorted(os.listdir(color_output_folder))

# Ensure filenames match
assert bw_files == color_input_files == color_output_files, "Filenames do not match!"

# Split files into training and validation set
train_ratio = 0.8
bw_train_files, bw_val_files, _, _ = train_test_split(bw_files, bw_files, test_size=1-train_ratio, random_state=42)

# Ensure filenames match for training and validation
assert set(bw_train_files) & set(bw_val_files) == set(), "Overlap between training and validation files!"

# Load a batch of images from a list of filenames and folder
def load_batch_from_folder(folder, files, start_idx, batch_size, color=True):
    images = []
    end_idx = min(start_idx + batch_size, len(files))
    
    for i in range(start_idx, end_idx):
        img_path = os.path.join(folder, files[i])
        if color:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        else:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
        images.append(img)
    return np.array(images)

# Separate generators for training and validation
def image_data_generator(files, batch_size):
    total_images = len(files)
    while True:
        for start_idx in range(0, total_images, batch_size):
            bw_batch = load_batch_from_folder(bw_folder, files, start_idx, batch_size, color=False)
            color_input_batch = load_batch_from_folder(color_input_folder, files, start_idx, batch_size)
            color_output_batch = load_batch_from_folder(color_output_folder, files, start_idx, batch_size)
            yield ([bw_batch, color_input_batch], color_output_batch)

# Example of usage
batch_size = 1
train_generator = image_data_generator(bw_train_files, batch_size)
val_generator = image_data_generator(bw_val_files, batch_size)

# for bw_batch, color_input_batch, color_output_batch in train_generator:
#     print('Train:', bw_batch.shape, color_input_batch.shape, color_output_batch.shape)

# for bw_batch, color_input_batch, color_output_batch in val_generator:
#     print('Validation:', bw_batch.shape, color_input_batch.shape, color_output_batch.shape)


In [29]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# batch_size = 32

# # Data Augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     zoom_range=0.15,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.15,
#     horizontal_flip=True,
#     fill_mode="nearest"
# )

# # Generator for augmented data
# def augmented_data_generator(batch_size):
#     base_generator = image_data_generator(batch_size)
    
#     for bw_batch, color_input_batch, color_output_batch in base_generator:
#         # Augment each batch
#         # Note: We're using the same seed for both black & white and color input images
#         # to ensure they undergo the same transformations.
        
#         # Augmenting BW images
#         bw_gen = datagen.flow(bw_batch, batch_size=batch_size, shuffle=False, seed=42)
        
#         # Augmenting color input images
#         color_input_gen = datagen.flow(color_input_batch, batch_size=batch_size, shuffle=False, seed=42)
        
#         # Augmenting color output images. Since we need to match outputs with inputs, 
#         # we're not shuffling and using a consistent seed.
#         color_output_gen = datagen.flow(color_output_batch, batch_size=batch_size, shuffle=False, seed=42)
        
#         yield [next(bw_gen), next(color_input_gen)], next(color_output_gen)

# # Example of usage
# aug_gen = augmented_data_generator(batch_size)
# for (bw_aug_batch, color_input_aug_batch), color_output_aug_batch in aug_gen:
#     print(bw_aug_batch.shape, color_input_aug_batch.shape, color_output_aug_batch.shape)


In [40]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Concatenate

def SimpleDualInput():
    # Black & white input
    bw_input = Input(shape=(2400, 1400, 1))
    # Colored reference input
    color_input = Input(shape=(2400, 1400, 3))

    # Merge inputs
    merge_layer = Concatenate()([bw_input, color_input]) # This will have 4 channels

    # Output layer to produce a 3-channel image
    outputs = Conv2D(3, (1, 1), activation='sigmoid')(merge_layer)

    return tf.keras.Model(inputs=[bw_input, color_input], outputs=outputs)

model = SimpleDualInput()
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


In [41]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [43]:
import math
steps_per_epoch = math.ceil(len(bw_train_files) / batch_size)
validation_steps = math.ceil(len(bw_val_files) / batch_size)

# 3. Train the model
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        epochs=1,
        verbose=1
    )

ResourceExhaustedError: Graph execution error:

Detected at node 'model_8/conv2d_51/Relu' defined at (most recent call last):
    File "C:\Python\anaconda3\envs\pipWindows_ml_color\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Python\anaconda3\envs\pipWindows_ml_color\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Python\anaconda3\envs\pipWindows_ml_color\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Python\anaconda3\envs\pipWindows_ml_color\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Python\anaconda3\envs\pipWindows_ml_color\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Python\anaconda3\envs\pipWindows_ml_color\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_fo